In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Mar 18 05:53:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
train_path = "/content/drive/MyDrive/KLTN/Dataset/act/csv/train.csv"
test_path = "/content/drive/MyDrive/KLTN/Dataset/act/csv/test.csv"

In [ ]:
train_df = pd.read_csv(train_path, delimiter="\t")
test_df = pd.read_csv(test_path, delimiter="\t")

In [ ]:
label_list = train_df.columns[1:]
label_list

Index(['Attraction-Inform', 'Attraction-NoOffer', 'Attraction-Recommend',
       'Attraction-Request', 'Attraction-Select', 'Booking-Book',
       'Booking-Inform', 'Booking-NoBook', 'Booking-Request',
       'Hospital-Inform', 'Hospital-Request', 'Hotel-Inform', 'Hotel-NoOffer',
       'Hotel-Recommend', 'Hotel-Request', 'Hotel-Select', 'Police-Inform',
       'Police-Request', 'Restaurant-Inform', 'Restaurant-NoOffer',
       'Restaurant-Recommend', 'Restaurant-Request', 'Restaurant-Select',
       'Taxi-Inform', 'Taxi-Request', 'Train-Inform', 'Train-NoOffer',
       'Train-OfferBook', 'Train-OfferBooked', 'Train-Request', 'Train-Select',
       'general-bye', 'general-greet', 'general-reqmore', 'general-thank',
       'general-welcome'],
      dtype='object')

In [ ]:
def get_labels(df):
    labels = []
    for i in range(len(df)):
        row = []
        for j in label_list:
            if ((j in df.columns) and (df.iloc[i][j] == 1)):
                row.append(1)
            else:
                row.append(0)
        labels.append(row)
    return labels

Y_test = get_labels(test_df)

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-05

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, is_test_df):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.text = self.df['utterance']
        if (is_test_df):
            self.labels = get_labels(self.df)
        else:
            self.labels = self.df[label_list].values

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'labels': torch.FloatTensor(self.labels[index])
        }

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifier, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

In [ ]:
def distance(tensor1, tensor2):
    list1 = tensor1.tolist()
    list2 = tensor2.tolist()

    """Distance between two vectors."""
    squares = [(p-q) ** 2 for p, q in zip(list1, list2)]
    return sum(squares) ** .5

In [ ]:
def contrastive_loss(batch, logits, temperature):
    contrastive_loss = 0
    for i in range(len(batch['input_ids'])):
        sum_labels = 0
        sum_distances = 0
        for k in range(len(batch['input_ids'])):
            if (i != k):
                sum_labels += torch.matmul(batch['labels'][i].t(), batch['labels'][k]).item()
                sum_distances += math.exp((-1)*distance(logits[i], logits[k])/temperature)
        for j in range(len(batch['input_ids'])):
            if (i != j):
                if (sum_labels == 0):
                    sum_labels = 1e-06
                contrastive_loss += (torch.matmul(batch['labels'][i].t(), batch['labels'][j]).item() / sum_labels) * math.log((math.exp((-1)*distance(logits[i], logits[j])/temperature)) / sum_distances)

    return (-1)*contrastive_loss


In [ ]:
def train(model, training_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in training_loader:
        input_ids = batch['input_ids'].to(device, dtype = torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.float)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask, token_type_ids)
        
        loss = criterion(logits, labels)
        # print('BCE Loss: ', loss)
        # print('Contrastive Loss: ', contrastive_loss(batch, logits, 1))
        loss += (0.01)*contrastive_loss(batch, logits, 10)
        # print('Loss: ', loss)
        # print("=======")

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(training_loader)

In [ ]:
def validate(model, validation_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in validation_loader:
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.float)

            logits = model(input_ids, attention_mask, token_type_ids)
            
            loss = criterion(logits, labels)
            loss += (0.01)*contrastive_loss(batch, logits, 10)

            total_loss += loss.item()
    return total_loss / len(validation_loader)

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)

            logits = model(input_ids, attention_mask, token_type_ids)

            predictions.append(torch.sigmoid(logits).cpu().detach().numpy())
    return np.concatenate(predictions)

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path)
    # Initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # Initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # Initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # Return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def save_checkpoint(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

**Main:**

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTClassifier(len(label_list))
model.to(device)

In [ ]:
# Initialize optimizer and loss function
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
# Load data
train_size = 0.8
train_df2 = train_df.sample(frac=train_size, random_state=200)
val_df = train_df.drop(train_df2.index).reset_index(drop=True)
train_df=train_df2.reset_index(drop=True)

train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN, False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=0)

valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN, False)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN, True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
# Train and validate the model
checkpoint_path = "/content/drive/MyDrive/KLTN/Model/BERT_CL/curr_checkpoint"
best_model_path = "/content/drive/MyDrive/KLTN/Model/BERT_CL/best_model.pt"
valid_loss_min = np.Inf

for epoch in range(EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    valid_loss = validate(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch + 1}: train_loss = {train_loss:.4f}, valid_loss = {valid_loss:.4f}')

    # Create checkpoint variable and add important data
    checkpoint = {
        'epoch': epoch + 2,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    # Save checkpoint
    save_checkpoint(checkpoint, False, checkpoint_path, best_model_path)

    # Save the model if validation loss has decreased
    if valid_loss <= valid_loss_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
      # Save checkpoint as best model
      save_checkpoint(checkpoint, True, checkpoint_path, best_model_path)
      valid_loss_min = valid_loss

Epoch 1: train_loss = 0.9979, valid_loss = 0.9408
Validation loss decreased (inf --> 0.940813).  Saving model ...
Epoch 2: train_loss = 0.8553, valid_loss = 0.8951
Validation loss decreased (0.940813 --> 0.895126).  Saving model ...
Epoch 3: train_loss = 0.8181, valid_loss = 0.8742
Validation loss decreased (0.895126 --> 0.874223).  Saving model ...


**Load model and continue training:**

In [ ]:
# Load the model
checkpoint_path = "/content/drive/MyDrive/KLTN/Model/BERT_CL/best_model.pt"

model, optimizer, epoch, loss = load_checkpoint(checkpoint_path, model, optimizer)

In [ ]:
print(epoch)
print(loss)

4
0.8742226623313528


In [ ]:
# Train and validate the model
checkpoint_path = "/content/drive/MyDrive/KLTN/Model/BERT_CL/curr_checkpoint"
best_model_path = "/content/drive/MyDrive/KLTN/Model/BERT_CL/best_model.pt"
valid_loss_min = loss

for e in range(epoch, EPOCHS + 2):
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    valid_loss = validate(model, valid_dataloader, criterion, device)
    print(f'Epoch {e}: train_loss = {train_loss:.4f}, valid_loss = {valid_loss:.4f}')

    # Create checkpoint variable and add important data
    checkpoint = {
        'epoch': e + 1,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    # Save checkpoint
    save_checkpoint(checkpoint, False, checkpoint_path, best_model_path)

    # Save the model if validation loss has decreased
    if valid_loss <= valid_loss_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
      # Save checkpoint as best model
      save_checkpoint(checkpoint, True, checkpoint_path, best_model_path)
      valid_loss_min = valid_loss

Epoch 4: train_loss = 0.7986, valid_loss = 0.8647
Validation loss decreased (0.874223 --> 0.864679).  Saving model ...
Epoch 5: train_loss = 0.7855, valid_loss = 0.8610
Validation loss decreased (0.864679 --> 0.861017).  Saving model ...
Epoch 6: train_loss = 0.7774, valid_loss = 0.8585
Validation loss decreased (0.861017 --> 0.858514).  Saving model ...


**Evaluation:**

In [ ]:
# Test the model and evaluate
from sklearn.metrics import accuracy_score, classification_report

threshold = 0.5

y_true = Y_test
y_pred = predict(model, test_dataloader, device)

for item in y_pred:
    for j in range(len(y_pred[0])):
        if (item[j] > threshold):
            item[j] = 1
        else:
            item[j] = 0

print("Test Accuracy : {}".format(accuracy_score(y_true, y_pred)))
print("\nClassification Report : ")
print(classification_report(y_true, y_pred, target_names=label_list))

Test Accuracy : 0.7777400976668475

Classification Report : 
                      precision    recall  f1-score   support

   Attraction-Inform       0.92      0.88      0.90      1522
  Attraction-NoOffer       0.92      0.80      0.86        60
Attraction-Recommend       0.71      0.66      0.69       148
  Attraction-Request       0.81      0.69      0.74       676
   Attraction-Select       0.64      0.53      0.58        55
        Booking-Book       0.89      0.97      0.93       537
      Booking-Inform       0.93      0.89      0.91       564
      Booking-NoBook       0.98      0.95      0.97       131
     Booking-Request       0.94      0.93      0.94       321
     Hospital-Inform       0.00      0.00      0.00         0
    Hospital-Request       0.00      0.00      0.00         0
        Hotel-Inform       0.89      0.89      0.89      2156
       Hotel-NoOffer       0.95      0.79      0.86        67
     Hotel-Recommend       0.70      0.70      0.70       140
       H

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1